In [0]:
create table demo.delta_lake.raw_stock_prices;

In [0]:
select * from demo.delta_lake.raw_stock_prices;

In [0]:
delete from demo.delta_lake.raw_stock_prices;

COPY INTO demo.delta_lake.raw_stock_prices
FROM 'abfss://demo@stgdeltadatabricks.dfs.core.windows.net/landing/stock_prices/'
FILEFORMAT = JSON
FORMAT_OPTIONS('inferedSchema' = 'true')
COPY_OPTIONS('mergeSchema' = 'true')

In [0]:
insert into demo.delta_lake.raw_stock_prices
select * from json.`abfss://demo@stgdeltadatabricks.dfs.core.windows.net/landing/stock_prices/stock_prices_day3.json`

In [0]:
select * from demo.delta_lake.raw_stock_prices

#### MERGE Command

In [0]:
create or replace table demo.delta_lake.stock_prices
(
  stock_id string,
  trading_date string,
  price double
)

In [0]:
merge into demo.delta_lake.stock_prices as tgt
using demo.delta_lake.raw_stock_prices as src
on tgt.stock_id = src.stock_id
when matched and src.status = 'ACTIVE' then
UPDATE set tgt.price = src.price,tgt.trading_date = src.trading_date
when matched and src.status = 'DELISTED' then
DELETE
when not matched and src.status = "ACTIVE" then
insert(price,trading_date,stock_id) values (src.price,src.trading_date,src.stock_id)

In [0]:
select * from demo.delta_lake.stock_prices

In [0]:
select * from demo.delta_lake.raw_stock_prices